## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

In [ ]:
%pip install okareo 

In [2]:
import os
from okareo import Okareo

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

response = okareo.upload_scenario_set(file_path='./webbizz_10_articles.jsonl', scenario_name="Retrieval Articles Scenario")
print(response) # use scenario set id in output for the next step

ScenarioSetResponse(scenario_id='2524063b-2faf-4762-8cfd-b1d6e02666f5', project_id='f7bfcff0-2b8d-4fec-ab20-51669cd3e732', time_created=datetime.datetime(2024, 1, 11, 0, 10, 10, 921425), type='SEED', tags=None, name='Retrieval Articles Scenario', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})


In [3]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioSetGenerate, ScenarioSetResponse, ScenarioType

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# Use scenario set id from previous step as source for generation 
response = okareo.generate_scenarios(
    source_scenario_id="899eae33-1ba7-4294-8f53-e705921ddb66",
    name="questions from retrieval articles",
    number_examples=2,
    generation_type=ScenarioType.TEXT_REVERSE_QUESTION
)

print(response)

ScenarioSetResponse(scenario_id='b1551494-7d58-49a2-a83d-47fdf049e9ba', project_id='f7bfcff0-2b8d-4fec-ab20-51669cd3e732', time_created=datetime.datetime(2024, 1, 11, 0, 10, 10, 982245), type='TEXT_REVERSE_QUESTION', tags=['seed:899eae33-1ba7-4294-8f53-e705921ddb66'], name='questions from retrieval articles', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})


In [4]:
# perform a test run using a scenario set loaded in the previous cell 
import os
import random
from datetime import datetime
from okareo import Okareo
from okareo_api_client.models import TestRunType
from okareo.model_under_test import CustomModel

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# Callable to be applied to each scenario in the scenario set
def call_model(input: str):
    # call your embedding model and vector db retrieval being tested here using <input> from the scenario set
    # we are using a random response here for demonstration purposes
    article_ids = [
        "75eaa363-dfcc-499f-b2af-1407b43cb133",
        "ac0d464c-f673-44b8-8195-60c965e47525",
        "35a4fd5b-453e-4ca6-9536-f20db7303344",
        "a8a97b0e-8d9a-4a1c-b93e-83d2bc9e5266",
        "0b85c12f-6ea6-4d4a-85de-6c6e9a9f8c78",
        "cda67f1d-19f2-4b45-9f3e-3b8d67f8c6c5",
        "6e4f1c97-3f7a-4fcd-a4a3-69c9817c8fd1",
        "f658c264-4a8a-4c93-a6d7-9a3d75f5a6f3",
        "aacf7a34-9d3a-4e2a-9a5c-91f2a0e8a12d",
        "f1a37b5e-58c4-4f5a-bc42-1b70253b8bf3"
    ]
    selected_ids = random.sample(article_ids, 5)   
    scores = [5, 4, 3, 2, 1]
    # higher score value means more relevant
    parsed_ids_with_scores = [(id, score) for id, score in zip(selected_ids, scores)]

    model_response =  {'matches': 'additional context from the model'} 

    # return a tuple of (parsed_ids_with_scores, overall model response context)
    return parsed_ids_with_scores, model_response

class RetrievalModel(CustomModel):
    def invoke(self, input: str):
        return call_model(input)

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="vectordb_retrieval_test", model=RetrievalModel(name="custom retrieval"))


today_with_time = datetime.now().strftime('%m-%d %H:%M:%S')
test_run_name=f"Retrieval Test Run {today_with_time}"

test_run_item = model_under_test.run_test(scenario=response, # use the scenario from the scenario set uploaded in the previous step
                                          name=test_run_name,
                                          test_run_type=TestRunType.INFORMATION_RETRIEVAL,
                                          calculate_metrics=True)

# display model level metrics for the test run
print(test_run_item.id)
print(test_run_item.model_metrics.to_dict())

08f3c168-38e7-4eaa-ba8d-64bbb970d725
{'Accuracy@k': {'1': 0.15, '2': 0.25, '3': 0.4, '4': 0.55, '5': 0.55, '6': 0.55, '7': 0.55, '8': 0.55, '9': 0.55, '10': 0.55}, 'Precision@k': {'1': 0.15, '2': 0.125, '3': 0.13333333333333336, '4': 0.1375, '5': 0.11000000000000001, '6': 0.09166666666666667, '7': 0.07857142857142856, '8': 0.06875, '9': 0.061111111111111116, '10': 0.05500000000000001}, 'Recall@k': {'1': 0.15, '2': 0.25, '3': 0.4, '4': 0.55, '5': 0.55, '6': 0.55, '7': 0.55, '8': 0.55, '9': 0.55, '10': 0.55}, 'NDCG@k': {'1': 0.15, '2': 0.21309297535714578, '3': 0.2880929753571458, '4': 0.3526944590681547, '5': 0.3526944590681547, '6': 0.3526944590681547, '7': 0.3526944590681547, '8': 0.3526944590681547, '9': 0.3526944590681547, '10': 0.3526944590681547}, 'MRR@k': {'1': 0.15, '2': 0.2, '3': 0.25, '4': 0.2875, '5': 0.2875, '6': 0.2875, '7': 0.2875, '8': 0.2875, '9': 0.2875, '10': 0.2875}, 'MAP@k': {'1': 0.15, '2': 0.2, '3': 0.25, '4': 0.2875, '5': 0.2875, '6': 0.2875, '7': 0.2875, '8': 0.2